In [ ]:
import pickle
from pathlib import Path
from collections import Counter
import pandas as pd
import gzip
import csv
import time
import hashed_index
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import importlib
importlib.reload(hashed_index)

<module 'hashed_index' from '/content/hashed_index.py'>

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
print(pv_name)
print(pv_temp)
print(pv_clean)


!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.



pageviews-202108-user.bz2
pageviews-202108-user-4dedup.txt
pageviews-202108-user.pkl
--2022-01-07 16:04:17--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

pageviews-202108-us 100%[===================>]   2.33G  4.57MB/s    in 9m 1s   

2022-01-07 16:13:19 (4.41 MB/s) - ‘pageviews-202108-user.bz2’ saved [2503235912/2503235912]



In [ ]:
path = 'drive/MyDrive/archive data/pr.csv.gz'
page_rank = {}
with gzip.open(path, "rt") as csvFile:
    csvreader = csv.reader(csvFile)
    for row in csvreader:
        page_rank.update({int(row[0]) :float(row[1])})

print(len(list(page_rank.items())))
print(list(page_rank.items())[:100])

6345849
[(3434750, 9913.728782160772), (10568, 5385.349263642036), (32927, 5282.081575765276), (30680, 5128.233709604118), (5843419, 4957.56768626387), (68253, 4769.278265355158), (31717, 4486.35018054831), (11867, 4146.4146509127695), (14533, 3996.4664408855015), (645042, 3531.6270898037424), (17867, 3246.0983906041406), (5042916, 2991.9457391661786), (4689264, 2982.324883041747), (14532, 2934.7468292031704), (25391, 2903.5462235134), (5405, 2891.4163291546365), (4764461, 2834.366987332661), (15573, 2783.8651181588375), (9316, 2782.0396464137707), (8569916, 2775.286191840017), (606848, 2773.576318485694), (14653, 2695.4293248287886), (18618239, 2450.113149559415), (19344515, 2347.0140777778715), (26667, 2224.080244382611), (23366462, 2143.5150505664656), (64065, 2127.7638023507975), (22936, 2127.075241657331), (26779, 2116.152456234464), (5407, 2099.6738846846106), (19827221, 2092.451347470643), (43432956, 2085.7409097922214), (679, 2065.030879305207), (61371, 1961.5232748658937), (21

In [ ]:
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    if int(parts[0]) in page_rank:
      wid2pv.update({int(parts[0]): int(parts[1])})

print(len(list(wid2pv.items())))
print(list(wid2pv.items())[:100])

6299824
[(7712754, 28713), (3632887, 528), (600744, 4285), (59804426, 61650), (4838455, 1450), (4822278, 779), (2556962, 594), (55029148, 123), (16381751, 209), (4700151, 1322), (5840274, 165), (1669534, 6340), (714570, 429), (594417, 7342), (1202168, 8311), (321544, 18535), (8355009, 64), (10936290, 27), (2978668, 288), (2349724, 4248), (4906617, 1997), (8004648, 93), (143804, 273), (1222755, 908), (4077845, 73), (5297162, 333), (1163095, 408), (14876253, 69), (9378013, 79), (6395043, 254), (1729032, 5426), (32566233, 2767), (47197315, 82), (5026730, 242), (221489, 8884), (10558683, 665), (1880521, 6984), (5886374, 1104), (22554980, 99194), (7198790, 391), (5260031, 397), (2597833, 646), (11934775, 54), (40042954, 3854), (3600326, 10684), (810047, 8062), (35473107, 6604), (5345208, 89), (20305346, 4402), (4189233, 424), (41274137, 14502), (712947, 174), (1921683, 344), (24976075, 287), (60432262, 134), (9179902, 30), (6893310, 122), (1283026, 3493), (2222596, 6870), (2516600, 2723), (

#Write Page View

In [ ]:
sort_dic = sorted(list(wid2pv.items()),key=lambda x: x[0])
print(sort_dic[:20])
print(sort_dic[-20:])


[(12, 75195), (25, 114746), (39, 24532), (290, 164494), (303, 111033), (305, 135191), (307, 343343), (308, 151041), (309, 2961), (316, 7167), (324, 223758), (330, 39), (332, 1199), (334, 5597), (336, 25818), (339, 126270), (340, 1222), (344, 1158), (358, 168853), (359, 6074)]
[(68380086, 32), (68380089, 36), (68380099, 87), (68380119, 363), (68380123, 179), (68380125, 30), (68380130, 39), (68380135, 56), (68380140, 20), (68380154, 11), (68380162, 69), (68380166, 613), (68380168, 70), (68380178, 75), (68380195, 30), (68380200, 27), (68380202, 61), (68380227, 29), (68380232, 8), (68380237, 19)]


In [ ]:
hashed_index.write_a_dictionary(sort_dic, 'drive/MyDrive/Test Data/pv/', 'pv' )

In [ ]:
t = time.time()
print(hashed_index.get_value('drive/MyDrive/Test Data/pv/', 'pv', 12))
print(hashed_index.get_value('drive/MyDrive/Test Data/pv/', 'pv', 68380086))
print(hashed_index.get_value('drive/MyDrive/Test Data/pv/', 'pv', 39))
print(time.time() - t)

75195
32
24532
0.012796640396118164


#Write Page Rank

In [ ]:
sort_dic = sorted(list(page_rank.items()),key=lambda x: x[0])
print(sort_dic[:20])
print(sort_dic[-20:])


[(12, 147.6714763991628), (25, 59.4435917259892), (39, 57.69149567519788), (290, 31.585455099103584), (303, 361.5932155448053), (305, 33.84140049129917), (307, 330.0459897512733), (308, 387.1322776955416), (309, 5.744656933511456), (316, 38.00746342354871), (324, 511.8274888202446), (330, 0.26669471586676635), (332, 4.11009348034522), (334, 21.551690485775747), (336, 31.272780124637023), (339, 36.03576746496808), (340, 3.349857303549437), (344, 6.882272039395503), (358, 483.89327617876563), (359, 0.6127222879316977)]
[(68380086, 0.1501208493870428), (68380089, 0.1501208493870428), (68380099, 0.1501208493870428), (68380119, 0.1501208493870428), (68380123, 0.1501208493870428), (68380125, 0.1501208493870428), (68380130, 0.1501208493870428), (68380135, 0.1501208493870428), (68380140, 0.1501208493870428), (68380154, 0.1501208493870428), (68380162, 0.1501208493870428), (68380166, 0.1501208493870428), (68380168, 0.1501208493870428), (68380178, 0.1501208493870428), (68380195, 0.150120849387042

In [ ]:
hashed_index.write_a_dictionary(sort_dic, 'drive/MyDrive/Test Data/pr/', 'pr' )

In [ ]:
t = time.time()
print(hashed_index.get_value('drive/MyDrive/Test Data/pr/', 'pr', 12))
print(hashed_index.get_value('drive/MyDrive/Test Data/pr/', 'pr', 68380237))
print(hashed_index.get_value('drive/MyDrive/Test Data/pr/', 'pr', 39))
print(time.time() - t)

147.6714763991628
0.1501208493870428
57.69149567519788
0.014164447784423828


In [ ]:
import os, os.path

files = [name for name in os.listdir('drive/MyDrive/Test Data/pr/')]
print(len(files))


files = [name for name in os.listdir('drive/MyDrive/Test Data/pv/')]
print(len(files))

2521
2521
